# A Transformer Learning to Speak

This notebook visualizes a character-level transformer model's training from random noise to something resembling Shakespeare texts.
Every 10 training iterations, the model generates one token, building a continuous stream of text
that reveals the gradual emergence of language understanding.

### Hyperparameters

In [1]:
import torch

batch_size = 64
block_size = 256

max_iters = 5_000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda'
eval_iters = 200
train_split = 0.9

n_embd = 384


In [2]:
n_head = 6
n_layer = 6
dropout = 0.2

generate_every = 10

### Data

In [3]:
with open('/data/tinyshakespeare.txt', 'r', encoding='utf-8') as file:
    text = file.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

n = int(train_split * len(data))
train_data, validation_data = data[:n], data[n:]


def get_batch(split):
    data = train_data if split == 'train' else validation_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

### Loss Estimation

In [4]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()

    for split in ['train', 'validation']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()

        out[split] = losses.mean()

    model.train()
    return out

### Model

In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(42)


class Head(nn.Module):
    """One self-attention head"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out


class MultiHeadAttention(nn.Module):
    """Multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    """Linear layer with ReLU activation"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd * 4),
            nn.ReLU(),
            nn.Linear(n_embd * 4, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """Transformer block: attention followed by feedforward"""

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            return logits, None

        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)
        return logits, loss

    @torch.no_grad()
    def generate_one(self, idx):
        """
        Generate a single next token given context.
        Returns the new token index and the updated context.
        """
        idx_cond = idx[:, -block_size:]
        logits, _ = self(idx_cond)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)
        return idx, idx_next.item()


model = BigramLanguageModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,} ({total_params / 1e6:.2f}M)")

Total parameters: 10,788,929 (10.79M)


### Training

Watch the transformer learn to speak. Every 10 iterations, it generates one character.
Early output will be random noise. As training progresses, patterns emerge:
first recognizable letters, then word-like structures, finally coherent Shakespeare.

In [6]:
from IPython.display import display, HTML, clear_output
import time

LINE_WIDTH = 100

def rgb_style(r, g, b):
    return f"color: rgb({r},{g},{b})"

def lerp_color(t, c1, c2):
    t = max(0, min(1, t))
    return tuple(int(c1[i] + t * (c2[i] - c1[i])) for i in range(3))

def progress_color(iteration, max_iter):
    t = iteration / max_iter
    start = (100, 100, 140)
    mid = (180, 200, 220)
    end = (255, 230, 180)
    if t < 0.5:
        color = lerp_color(t * 2, start, mid)
    else:
        color = lerp_color((t - 0.5) * 2, mid, end)
    return color

def loss_color(loss, min_loss=0.5, max_loss=2.8):
    t = 1 - (loss - min_loss) / (max_loss - min_loss)
    t = max(0, min(1, t))
    if t < 0.5:
        r, g = 255, int(255 * t * 2)
    else:
        r, g = int(255 * (1 - t) * 2), 255
    return (r, g, 80)

generation_context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = ""
loss_history = []
current_line = ""
current_loss = 4.5

# Store all output as HTML for final display
all_lines_html = []
current_iteration = 0

def escape_html(text):
    return text.replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;").replace(" ", "&nbsp;")

def render_current_state():
    """Re-render the entire output display."""
    # Build HTML for completed lines
    completed = "".join(all_lines_html)

    # Build HTML for current line in progress
    if current_line:
        text_color = progress_color(current_iteration, max_iters)
        l_color = loss_color(current_loss)
        escaped = escape_html(current_line)
        padding = "&nbsp;" * max(0, LINE_WIDTH - len(current_line))
        current_html = (
            f'<span style="{rgb_style(*text_color)}">{escaped}</span>'
            f'{padding}'
            f'<span style="opacity:0.5">│</span> '
            f'<span style="{rgb_style(*l_color)}">{current_iteration:>5}</span> '
            f'<span style="opacity:0.5">·</span> '
            f'<span style="{rgb_style(*l_color)}">{current_loss:.2f}</span>'
        )
    else:
        current_html = ""

    header = f'''
<div style="font-family: monospace; white-space: pre; line-height: 1.4; background: #1e1e1e; padding: 10px; color: #ccc;">
<span style="font-weight:bold">{"═" * 120}</span>
<span style="font-weight:bold">  Watch a Transformer Learn to Shakespeak</span>
<span style="font-weight:bold">{"═" * 120}</span>
<span style="opacity:0.6">  Training: {max_iters} iterations │ Generating: 1 token every {generate_every} steps</span>
<span style="opacity:0.6">  Expected output: ~{max_iters // generate_every} characters</span>
<span style="font-weight:bold">{"─" * LINE_WIDTH}</span> <span style="opacity:0.5">│  iter · loss</span>

{completed}{current_html}
</div>
'''
    clear_output(wait=True)
    display(HTML(header))

def finalize_line(line_text, iteration, loss):
    """Convert current line to final HTML and add to completed lines."""
    text_color = progress_color(iteration, max_iters)
    l_color = loss_color(loss)
    escaped = escape_html(line_text)
    padding = "&nbsp;" * max(0, LINE_WIDTH - len(line_text))
    line_html = (
        f'<span style="{rgb_style(*text_color)}">{escaped}</span>'
        f'{padding}'
        f'<span style="opacity:0.5">│</span> '
        f'<span style="{rgb_style(*l_color)}">{iteration:>5}</span> '
        f'<span style="opacity:0.5">·</span> '
        f'<span style="{rgb_style(*l_color)}">{loss:.2f}</span>\n'
    )
    all_lines_html.append(line_html)

# Initial render
render_current_state()

for iter in range(max_iters):
    current_iteration = iter

    if iter % eval_interval == 0:
        losses = estimate_loss()
        current_loss = losses['train'].item()
        loss_history.append((iter, losses['train'].item(), losses['validation'].item()))

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    current_loss = 0.99 * current_loss + 0.01 * loss.item()

    if iter % generate_every == 0:
        model.eval()

        if generation_context.shape[1] > block_size:
            generation_context = generation_context[:, -block_size:]

        generation_context, new_token_idx = model.generate_one(generation_context)
        new_char = itos[new_token_idx]
        generated_text += new_char

        if new_char == '\n':
            finalize_line(current_line, iter, current_loss)
            current_line = ""
        else:
            current_line += new_char
            if len(current_line) >= LINE_WIDTH:
                finalize_line(current_line, iter, current_loss)
                current_line = ""

        render_current_state()
        model.train()

if current_line:
    finalize_line(current_line, max_iters, current_loss)
    render_current_state()

losses = estimate_loss()
loss_history.append((max_iters, losses['train'].item(), losses['validation'].item()))

# Final footer
clear_output(wait=True)
footer_html = f'''
<div style="font-family: monospace; white-space: pre; line-height: 1.4; background: #1e1e1e; padding: 10px; color: #ccc;">
<span style="font-weight:bold">{"═" * 120}</span>
<span style="font-weight:bold">  Watch a Transformer Learn to Shakespeak</span>
<span style="font-weight:bold">{"═" * 120}</span>
<span style="opacity:0.6">  Training: {max_iters} iterations │ Generating: 1 token every {generate_every} steps</span>
<span style="opacity:0.6">  Expected output: ~{max_iters // generate_every} characters</span>
<span style="font-weight:bold">{"─" * LINE_WIDTH}</span> <span style="opacity:0.5">│  iter · loss</span>

{"".join(all_lines_html)}
<span style="font-weight:bold">{"═" * 120}</span>
<span style="font-weight:bold">  [End of Emergence]</span>
<span style="font-weight:bold">{"═" * 120}</span>
</div>
'''
display(HTML(footer_html))